<h1 align="center">Cloud Computing</h1>

<a id='Distributed Systems'></a>
<h3 style="background-color:#616161;color:white">1. Distributed Systems</h3>

<h4>Lamport Timestamps</h4>

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud001.png" height="100" width="500">

How the timestamp (i.e. integer incrementing) works:
* Everything is set to 0 to begin with
* Everything can do 3 things: Internal action, Send, & Recieve.
* Whenver internal or send happens it increments its own timestamp by 1
* Whenever something recieves something it takes the max of its own timestamp and that of the recieved message, and increments that by 1 to get to a new timestam number for itself

Notes:
* Does not allow global ordering of event
* Lamport lacks Gap Detection - cannot tell if an event exists elsewhere that sits between two events
* Can however assume that if an event from a comes before event from b, then a's clock is lower than b's clock (Known as the Clock consistency condition)
* The strong clock consistency condition assumes this and vice-versa

<h4> Deadlocks</h4>

* A deadlock is a situation in which two or more competing actions are each waiting for the other to finish, and thus neither ever does
* The Cauffman conditions for a deadlock to occur are:
- Mutual Exclusion: There is a resource that can only be used by one process at a time
- Hold and Wait: There is a process that needs to hold a resource and wait for others
- No Preemption: The monitor cannot forve a process to give up a resource
- Circular Wait: A waits for B who waits for C who waits for A

<h4>Livelock</h4>
Similar to a deadlock, except that the states of the processes involved constantly change with regard to one another, none progressing

<h4>The Cap Theorem</h4>

Published by Eric Brewer in 2000, the theorem is a set of basic requirements that describe any distributed system. If you imagine a distributed database system with multiple servers, here's how the CAP theorem applies:

* Consistency - All the servers in the system will have the same data so users will get the same copy regardless of which server answers their request.
* Availability - The system will always respond to a request (even if it's not the latest data or consistent across the system or just a message saying the system isn't working).
* Partition Tolerance - The system continues to operate as a whole even if individual servers fail or can't be reached.

It's theoretically impossible to have all 3 requirements met, so a combination of 2 must be chosen and this is usually the deciding factor in what technology is used.

When it comes to distributed databases, the two choices are only AP or CP because if it's not partition tolerant, it's not really a reliable distributed database. So the choice is simpler: if a network split happens, do you want the database to keep answering but with possibly old/bad data (AP)? Or should it just stop responding unless you can get the absolute latest copy (CP)?

<h4>Dealing with failures:</h4>

* Many cloud computations run for extended periods of time on multiple servers.
* Checkpoints are taken periodically in anticipation of the need to restart a process when one or more
systems fail.
* When a failure occurs, the computation is restarted from the last checkpoint rather than from the beginning.
* Intuitively, the construction of the global state is equivalent to taking snapshots of individual processes and then combining these snapshots into a global view.

* Yet, combining snapshots is straightforward if and only if all processes have access to a global clock and the snapshots are taken at the same time

<b>Chandy-Lamport Algorithm</b>

1. The observer process (the process taking a snapshot):  
    * Saves its own local state
    * Sends a snapshot request message bearing a snapshot token to all other processes
2. A process receiving the snapshot token for the first time on any message: 
    * Sends the observer process its own saved state
    * Attaches the snapshot token to all subsequent messages (to help propagate the snapshot token)
3. When a process that has already received the snapshot token receives a message that does not bear the snapshot token, this process will forward that message to the observer process. This message was obviously sent before the snapshot “cut off” (as it does not bear a snapshot token and thus must have come from before the snapshot token was sent out) and needs to be included in the snapshot.

From this, the observer builds up a complete snapshot: a saved state for each process and all messages “in the ether” are saved

<h4>Crash Failures</h4>
1. Crash failure: the process abruptly stops and does not resume
2. Byzantine failure: the process fail in arbitrary ways (due to malfunctioning hardware, buggy software, malicious attacks from hackers, etc.)
    * Omission failures, e.g., failing to receive a request
    or send a response.
    * Commission failures, e.g., corrupting the local state, sending an incorrect/inconsistent response to the request.

Of the two types of failures,Byzantine failures are far more disruptive

<h4>Paxos Consensus Protocol</h4>

The basic Paxos protocol assumes
– Processes run on processors and communicate
through a network;
processors and network may experience failures,
but not Byzantine failures.

The Processors
* operate at arbitrary speeds;
* have stable storage and may rejoin the protocol after a failure;
* send messages to one another

The network:
* may lose, reorder, or duplicate messages;
* messages are sent asynchronously; it may take arbitrary long time to reach the destination

The basic Paxos protocol has two phases.

Phase I
* Proposal preparation
* Proposal promise

Phase II
* Accept request
* Accepted

The benefit of the Paxos protocols is the
guarantee of three safety properties
* Non-triviality: Only proposed values can be
learned.

* Consistency: at most one value can be learned.

* Liveness: if a value v has been proposed,
eventually every learner will learn some value,
provided that sufficient processors remain nonfaulty.


<a id='Distributed Systems'></a>
<h3 style="background-color:#616161;color:white">2. Map-Reduce</h3>

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud002.png" height="100" width="500">



<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud003.png" height="100" width="400">

<b>Partitioning:</b>
* Used to ensure records with the same key end up at the same worker
* Uses a simple function hash(key) mod R to decide which Reducer to go to
* Sometimes useful to override. E.g hash(hostname(URL)) mod R ensures URLs from one host end up in the same output file
    
* MapReduce program in Hadoop = Hadoop job

<h4> Combiners </h4>

This mapper function which has a multii-doc wrapper for a single doc function exemplifies the need for a combiner in a word count of individual documents

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud005.png" height="100" width="400">

* The combiner could group together counts for the same document
* While the combiner can do a similar role to the reducer, bear in mind the combiner only gets a subset of the data, so if you were trying to calculate averages for example you'd end calculating a mini average and pushing that forward to the reducer

http://www.philippeadjiman.com/blog/2010/01/14/hadoop-tutorial-series-issue-4-to-use-or-not-to-use-a-combiner/

<b>Calculating a co-occurence matrix</b>

Pairs:


<a id='MapReduce Optimization'></a>
<h3 style="background-color:#616161;color:white">4. MapReduce Optimization</h3>


<b>Ways to optimize:</b>
* Compression of data from one flow to the next
    * Trade-off between the volume of data transfer and the (de)compression time.
    * Usually, compressing map outputs using a fast compressor increases efficiency

* Available memory
    * Particularly for Reduce Shuffle and Sort

* Speculative Execution
    * Jobtracker tries to detect tasks that are taking a long time, such as one node being very slow
    * In this case it would make copies of the tasks and execute them
    * Whichever one finishes first wins

<a id='AlgoDesign'></a>
<h3 style="background-color:#616161;color:white">5. Map-Reduce Algorithm Design</h3>

* <b>“In-Mapper Combining”</b>
    * The in-mapper combiner takes this optimization a bit further: the aggregations do not even write to local disk: they occur in-memory in the Mapper itself.
    * So one example is to have an Initialize, Map, an Call method in your Mapper, where the intialize setups an an associative array to store results, the mappers does each indicidual processing and stores in the array, and close does an aggregation before emitting

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud005.png" height="100" width="400">

Advatages: Speed as not writing to disk
Disadvantages: Explicit memory management required. Potential for order-dependent bugs

* <b>“Pairs” vs “Stripes”</b>
 * Pairs is just emitting a key and a value
 * Stripes is grouping multiple results into an associative array with a defined structure so thaat in the combiner or reduce phase it can efficiently interpret the results
 
* <b>Secondary Sorting</b>
    * Value-to-Key Conversion: Creating a composite key is straight forward. What we need to do is analyze what part(s) of the value we want to account for during the sort and add the appropriate part(s) to the natural key. 

<a id='Hadoop'></a>
<h3 style="background-color:#616161;color:white">3. Hadoop</h3>

* SequenceFiles: Binary encoded of a sequence of key/value pairs

* Concrete classes for different data types:
    * IntWritable
    * LongWritable
    * Text
    * etc.

* Writable Defines a de/serialization protocol.
* Every data type in Hadoop is a Writable.
* WritableComparable Defines a sort order.
* All keys must be of this type (but not values).


<b>Job submission process</b>
* Client (i.e., driver program) creates a job, configures it, and submits it to job tracker
* JobClient computes input splits (on client end)
* Job data (jar, configuration XML) are sent to JobTracker
* JobTracker puts job data in shared location, enqueues tasks
* TaskTrackers poll for tasks
* Off to the races…

<h4>Hadoop Map-Reduce Process</h4>
<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud004.png" height="100" width="400">

* Map outputs are buffered in memory in a circular
buffer
* When buffer reaches threshold, contents are
“spilled” to disk
* Spills merged in a single, partitioned file (sorted
within each partition): combiner runs here


Reduce side
* First, map outputs are copied over to reducer
machine
* “Sort” is a multi-pass merge of map outputs
(happens in memory and on disk): combiner runs
here
* Final merge pass goes directly into reducer

<a id='Other'></a>
<h3 style="background-color:#616161;color:white">5. Other</h3>

HTTP Requests:
* GET method should be safe (nullipotent) - calling it produces no side effects (unlike PUT and GET which may change things)
* The GET, PUT and DELETE methods should be idempotent - repeated calls should have the same effect as a single request 
* Post is not idempotent

<a id='Q&A'></a>
<h3 style="background-color:#616161;color:white">4. Q&A</h3>

<h4>
There is a large text file of information about films stored in the HDFS over a number of
machines. Each line of this file describes the details of one film in the following format.
title | year | runtime | genres | stars
<br><br>
The different fields are separated by the | character; the list of genres and the list of stars
are both separated by commas; the runtime is measured in minutes.
An example line is given below.
The Godfather | 1972 | 175 | Crime, Drama | Marlon Brando, Al Pacino, James Caan
You can assume that there are no duplicate records, and each distinct star’s name is unique.
Write a MapReduce program (in pseudo-code) to calculate for each genre the average
runtime of film in the 1970s.
A combiner should be implemented to accelerate the computation.
</h4>

<code>
class Mapper
    method Map{string t, int r}
        s = split s based on |
        g = get genre from s
        r = get run time from s
        Emit(string g, pair (r, 1))     # Emit genre and (runtime , 1)

clase Combiner
    method Combine(string g, pairs[(r1,c1)(r2, c2) ...] P)
    sum = 0
    count = 0
    for each pair (r,c) in P
        sum = sum + r
        count = count + c
    Emit (g,pair(sum,count))

class Reducer
    method Reducer
</code>